# Transformada de Fourier discreta (DFT)

A Transformada de Fourier decompõe qualquer sinal em uma soma de ondas cosseno e seno, de forma que pode-se medir a frequência, a amplitude e a fase. Neste notebook, vamos trabalhar com sinais discretos, usando tanto a Transformada de **Fourier Discreta (DFT)** quanto a **Transformada de Fourier Rápida (FFT)**.

Considerando um sinal cujas amostras estão igualmente espaçadas, a DFT é definida como:

$\large \displaystyle{X_k = \sum\limits_{n=0}^{N-1}x_n \cdot e^{-i2\pi kn/N}}$

onde $N$ é o número de amostras, $n$ é o índice da amostra atual, $k$ é a frequência atual ($k \in [0, N-1]$), $x_n$ é o valor da amostra em $n$ e $X_k$ é a DFT. Perceba que:

- Se N for ímpar, $X_1$, $X_2$, $\dots$, $X_{(N-1)/2}$ contém os termos de frequência positiva e $X_{(N+1)/2}$, $\dots$, $X_{N-1}$ contém os termos de frequência negativa.
- Se N for par, $X_1$, $X_2$ $\dots$, $X_{N/2 - 1}$ contém os termos de frequência positiva e $X_{N/2}$, $\dots$, $X_{N-1}$ contém os termos de frequência negativa.
- No caso de um sinal de entrada real, a parte das frequências positivas da DFT é o conjugado da parte negativa, e assim o espectro vai ser simétrico. Portanto, geralmente plotamos apenas a DFT correspondendo às frequências positivas.

A amplitude e a fase do sinal podem ser calculadas por:

$\large A = \frac{|X_k|}{N} = \frac{\sqrt{Re(X_k)^2 + Im(X_k)^2}}{N} $

$\large \Phi = atan2(Im(X_k), Re(X_k))$

As amplitudes retornadas pela DFT vão corresponder às amplitudes do sinal original se nós normalizarmos pelo número de amostras. Entretanto, se nós olharmos apenas para um lado do resultado da DFT, nós dividimos por N/2, e não por N, para obter a amplitude.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fftfreq

In [ ]:
T = 40 # segundos
N = 100 # amostras
t = np.linspace(0, T, N)
dt = t[1] - t[0]

Dadas essas informações, a frequência de Nyquist é:

In [ ]:
f_ny = 1/(2*dt)
f_ny

Vamos olhar para algumas frequências.

In [ ]:
f1 = 20/(N*dt)
f2 = 10/(N*dt)

In [ ]:
round(f1, 4)

In [ ]:
round(f2, 4)

Obtendo uma série temporal com ruído incluso.

In [ ]:
x1 = np.sin(2*np.pi*f1*t) + 0.3*np.sin(2*np.pi*f2*t) + 0.3*np.random.randn(len(t))

In [ ]:
plt.plot(t, x1)
plt.xlabel("$t (s)$")
plt.ylabel("Amplitude");

Primeiramente, vamos definir uma função simples de DFT e usar funções prontas do `Numpy` apenas para criar o vetor de frequências.

In [ ]:
def DFT(x):
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(-2j * np.pi * k * n / N)
    
    X = np.dot(e, x)
    
    return X

In [ ]:
# DFT
X = DFT(x1)
# vetor de frequências
f = np.fft.fftfreq(len(t), dt)

In [ ]:
# como o sinal de entrada é real, vamos plotar apenas a primeira metade
plt.plot(f[:N//2], np.abs(X[:N//2]))
plt.xlabel("$f_n (s^{-1})$")
plt.ylabel("$|X_k|$");

Para exemplificar a questão da amplitude, vamos normalizar o sinal.

In [ ]:
X_norm = X[:N//2]/(N//2)
plt.plot(f[:N//2], np.abs(X_norm))
plt.title("DFT normalizada")
plt.xlabel("$f_n (s^{-1})$")
plt.ylabel("$|X_k|$");

Agora pode ser visto que os dois picos recuperaram as amplitudes de 1.0 e 0.3 do sinal original, mesmo com o ruído. Antes de prosseguirmos para a FFT, vamos verificar o tempo de execução da nossa implementação da DFT.

In [ ]:
%timeit DFT(x1)

# Transformada de Fourier rápida (FFT)

A Transformada de Fourier rápida consiste em um algorimo para calcular a DFT de uma sequência de forma eficiente. Foi descrita primeiramente em um artigo de Cooley e Tukey em 1965, e é um algoritmo de divisão e conquista que quebra a DFT recursivamente em DFTs menores e reduz a complexidade do problema de $O(n^2)$ para $O(nlogn)$, onde n é o tamanho dos dados.

A FFT utiliza de simetrias na DFT para acelerar o seu cálculo. Com base na definição da DFT apresentada anteriormente, vamos calcular o seguinte:

$\large \displaystyle{X_{k+N} = \sum\limits_{n=0}^{N-1}x_n \cdot e^{-i2\pi (k+N)n/N}} = \sum\limits_{n=0}^{N-1}x_n \cdot e^{-i2\pi n} \cdot e^{-i2\pi kn/N} $

Note que $e^{-i\pi n} = 1$, portanto:

$\large \displaystyle{X_{k+N} = \sum\limits_{n=0}^{N-1}x_n \cdot e^{-i2\pi kn/N} = X_k} $

e, de forma geral

$X_k = X_{k \pm N} = X_{k \pm 2N} = \dots = X_{k \pm cN}$ para todo inteiro $c$

Cooley e Tukey mostraram que podemos calcular a DFT de forma mais eficiente se dividirmos o problema em problemas menores. Vamos dividir a série temporal em duas:

$\large \displaystyle{X_k = \sum\limits_{n=0}^{N-1}x_n \cdot e^{-i2\pi kn/N}} \\ \large = \sum\limits_{m=0}^{N/2-1}x_{2m}\cdot e^{-i2\pi k(2m)/N} + \sum\limits_{m=0}^{N/2-1} x_{2m+1}\cdot e^{-i2\pi k(2m+1)/N} \\ \large = \sum\limits_{m=0}^{N/2-1} x_{2m} \cdot e^{-i2\pi km/(N/2)} + e^{-i2\pi k/N} + \sum\limits_{m=0}^{N/2-1}x_{2m+1} \cdot e^{-i2\pi km/(N/2)}$

Nota-se que os dois termos menores que possuem apenas metade do tamanho $\frac{N}{2}$ são duas DFTs menores. Para cada termo, $0 \le m \le \frac{N}{2}$, mas $0 \le k \le N$, e metade dos valores serão os mesmos devido à simetria discutida anteriormente. Não há necessidade de parar por aqui, e poderíamos continuar dividindo cada termo pela metade até chegarmos aos últimos dois termos, que é a implementação do `Numpy`faz.

Vamos agora importar a implementação da FFT do `Numpy` e recalcular o exemplo anterior.

In [ ]:
from numpy.fft import fft

In [ ]:
X = fft(x1)

In [ ]:
plt.plot(f[:N//2], np.abs(X[:N//2]))
plt.xlabel("$f_n (s^{-1})$")
plt.ylabel("$|X_k|$");

Vamos verificar o tempo de execução

In [ ]:
%timeit fft(x1)

Como pode ser visto, houve uma redução drástica nos tempos de execução ao comparar a DFT e a implementação de FFT. Vamos continuar usando a FFT, definir novos sinais, e realizar alguns testes.

In [ ]:
f2 = 10/(N*dt)
f3 = (10+5*N)/(N*dt)

In [ ]:
round(f2, 4)

In [ ]:
round(f3, 4)

In [ ]:
x2 = np.sin(2*np.pi*f2*t) + 0.1*np.random.randn(len(t))
x3 = np.sin(2*np.pi*f3*t) + 0.1*np.random.randn(len(t))

In [ ]:
plt.plot(t, x2, label="f2")
plt.plot(t, x3, label="f3")
plt.legend()
plt.xlabel("$t(s)$")
plt.ylabel("Amplitude");

Perceba que a frequência f3 é bem mais alta que a frequência f2, mas ao plotarmos, elas parecem a mesma. Por quê? Vamos computar a FFT dos sinais.

In [ ]:
x2_FFT = fft(x2)
x3_FFT = fft(x3)

In [ ]:
plt.plot(f[:N//2], np.abs(x2_FFT[:N//2]), label="$X_2$")
plt.plot(f[:N//2], np.abs(x3_FFT[:N//2]), 'r--', label="$X_3$")
plt.axvline(f_ny, ls="--", color="k")
plt.xlabel("$f_n (s_{-1})$")
plt.legend()
plt.ylabel("$|X_k|$");

Os gráficos das transformadas de Fourier dos dois sinais parecem os mesmos. Isso acontece pois a amostragem adotada não foi capaz de recuperar o sinal com mais de 12 Hz, e ele foi "colapsado" em um pico de frequência bem menor, ou seja, ocorreu **aliasing** (a frequência de Nyquist é indicada pela reta preta).